<a href="https://colab.research.google.com/github/Lursios/projects/blob/main/College_Related_Projects/Ship_Production_Optimization_Class/Particle_Swarm_UAS_Optimasi_Produksi_Kapal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import queue

In [ ]:
#Swarm Intialization
n_pop = 25 #Jumlah partikel
n_blocks = 8
n_workstation = 6
#Xp = np.array([np.random.permutation(range(1,n_blocks + 1)) for _ in range(n_pop)]) # Array of particle need connversion
X = np.random.uniform(0, n_blocks, size= (n_pop,n_blocks))
V = np.random.uniform(0, n_blocks, size= (n_pop,n_blocks))

#data for fitness evaluation
sample_prep_time = np.random.randint(n_blocks, size = (n_blocks,n_workstation))
sample_process_time = np.random.randint(n_blocks, size = (n_blocks,n_workstation))

#liat datanya
print(sample_process_time )

[[0 4 1 3 2 6]
 [1 5 1 0 7 7]
 [5 4 5 6 5 1]
 [2 2 4 0 4 1]
 [7 5 3 7 4 0]
 [3 4 4 1 5 6]
 [3 0 3 1 1 0]
 [5 7 5 4 3 0]]


In [ ]:
#def encoding_pop(solutions,reverse = False):
 # X_encoded = []
 # for solution in solutions :
 #   if reverse == True:
 #     temp = {}
 #     encoded = []
 #     rand_nums = [np.random.uniform(0,n_blocks) for i in range(solution)]
 #   for reversal in np.argsort(rand_nums):
 #       temp[reversal] = rand_nums[reversal]
 #     for value in solution :
 #       encoded.append(temp[value])
 #     X_encoded.append(encoded)
 #   else :
 #       X_encoded.append(np.argsort(solution) + 1)
 #   return np.array(X_encoded)

#print(encoding_pop(Xp,True))



In [ ]:
#Hyperparameter
w = 0.5 #Interial Weight
c1 = 0.3  #Cognitive Learning Factor
c2 = 0.3  #Social Leraning Factor

#Conversion Based on Ranked Order Value
def encoding_pop(solutions,reverse = True):
  X_encoded = []
  for solution in solutions :
      X_encoded.append(np.argsort(solution) + 1)
  return np.array(X_encoded)

def encoding_part(solution) :
  return np.argsort(solution) + 1

#contoh printan
#print(X)
#print(encoding(X))


In [ ]:
#Fitness Evaluation
def calculate_obj(sol,sample_process_time,n_blocks,m_station):
  qTime = queue.PriorityQueue()
  qMachines = []
  for i in range(m_station):
      qMachines.append(queue.Queue())

  for i in range(n_blocks):
      qMachines[0].put(sol[i])

  busyMachines = []
  for i in range(m_station):
      busyMachines.append(False)

  time = 0
  job = qMachines[0].get()
  qTime.put((time + sample_process_time[job-1][0], 0, job))
  busyMachines[0] = True

  while True:
          time, workstation , block = qTime.get()
          #print(f"it's processing block {block} at time interval {time} in workstasion {workstation+1} current work stasion condition : {busyMachines}")
          # Remove the hashtage on upper code to see the calculation processes
          if block == sol[n_blocks-1] and workstation == m_station-1:
              break
          busyMachines[workstation] = False
          if not qMachines[workstation].empty():
                  j = qMachines[workstation].get()
                  qTime.put((time+sample_process_time[j-1][workstation], workstation, j))
                  busyMachines[workstation] = True
          if workstation < m_station-1:
              if busyMachines[workstation+1] == False:
                  qTime.put((time+sample_process_time[block-1][workstation], workstation+1, block))
                  busyMachines[workstation+1] = True
              else:
                  qMachines[workstation+1].put(block)
  return time

def fitness(converted_solutions,cal_obj,process_time,n_blocks,n_workstation):
  c_max = [] #Maximum Block Completion Time
  for solution in converted_solutions :
    c_max.append(calculate_obj(solution,process_time,n_blocks,n_workstation))
  return np.array(c_max)




In [ ]:
# Initialize data
pbest = X #Convert dulu jadi location
pbest_obj = fitness(encoding_pop(X),calculate_obj,sample_process_time,n_blocks,n_workstation)
gbest = pbest[pbest_obj.argmin()]
gbest_obj = pbest_obj.min()

print(encoding_part(gbest))
print(gbest)
print(gbest_obj)

[2 4 3 6 1 5 8 7]
[6.35930897 0.34415581 2.22596698 1.43157964 6.79932061 5.3930378
 7.75193268 7.26933721]
52


In [ ]:
#Updating Particle
def update():
    "Function to do one iteration of particle swarm optimization"
    global V, X, pbest, pbest_obj, gbest, gbest_obj
    # Update params
    r1, r2 = np.random.rand(2)
    V = w * V + c1*r1*(pbest - X) + c2*r2*(gbest.reshape(-1,n_blocks)-X)
    Xn = X + V
    obj = fitness(encoding_pop(Xn),calculate_obj,sample_process_time,n_blocks,n_workstation)
    pbest[(pbest_obj >= obj)] = X[(pbest_obj >= obj)]
    pbest_obj = np.array([pbest_obj, obj]).min(axis=0)
    gbest = pbest[pbest_obj.argmin()]
    gbest_obj = pbest_obj.min()

In [ ]:
#Stopping Condition
iteration = 0
max_iter = 300
while True :
  if iteration == max_iter :
    print(f"PSO found best schedule at {gbest} with total time {gbest_obj} at {iteration} iteration")
    break
  update()
  iteration += 1

PSO found best schedule at [5.4476879  5.95328952 7.70881787 6.81896899 2.54884089 2.00489169
 5.76238691 6.73962077] with total time 49 at 300 iteration


In [ ]:
#Visualization
#Draw a Gantt chart
def Gantt(self):
    fig = plt.figure()
    M = ['red', 'blue', 'yellow', 'orange', 'green', 'moccasin', 'purple', 'pink', 'navajowhite', 'Thistle',
          'Magenta', 'SlateBlue', 'RoyalBlue', 'Aqua', 'floralwhite', 'ghostwhite', 'goldenrod', 'mediumslateblue',
          'navajowhite','navy', 'sandybrown']
    M_num=0
    for i in range(len(self.M)):
        for j in range(self.M[i]):
            for k in range(len(self.Machines[i][j].start)):
                Start_time=self.Machines[i][j].start[k]
                End_time=self.Machines[i][j].end[k]
                Job=self.Machines[i][j]._on[k]
                plt.barh(M_num, width=End_time - Start_time, height=0.8, left=Start_time, \
                          color=M[Job], edgecolor='black')
                plt.text(x=Start_time + ((End_time - Start_time) / 2 - 0.25), y=M_num - 0.2,
                          s=Job+1, size=15, fontproperties='Times New Roman')
            M_num += 1
    plt.yticks(np.arange(M_num + 1), np.arange(1, M_num + 2), size=20, fontproperties='Times New Roman')

    plt.ylabel("machine", size=20, fontproperties='SimSun')
    plt.xlabel("time", size=20, fontproperties='SimSun')
    plt.tick_params(labelsize=20)
    plt.tick_params(direction='in')
    plt.show()


[0.         0.05050505 0.1010101  0.15151515 0.2020202  0.25252525
 0.3030303  0.35353535 0.4040404  0.45454545 0.50505051 0.55555556
 0.60606061 0.65656566 0.70707071 0.75757576 0.80808081 0.85858586
 0.90909091 0.95959596 1.01010101 1.06060606 1.11111111 1.16161616
 1.21212121 1.26262626 1.31313131 1.36363636 1.41414141 1.46464646
 1.51515152 1.56565657 1.61616162 1.66666667 1.71717172 1.76767677
 1.81818182 1.86868687 1.91919192 1.96969697 2.02020202 2.07070707
 2.12121212 2.17171717 2.22222222 2.27272727 2.32323232 2.37373737
 2.42424242 2.47474747 2.52525253 2.57575758 2.62626263 2.67676768
 2.72727273 2.77777778 2.82828283 2.87878788 2.92929293 2.97979798
 3.03030303 3.08080808 3.13131313 3.18181818 3.23232323 3.28282828
 3.33333333 3.38383838 3.43434343 3.48484848 3.53535354 3.58585859
 3.63636364 3.68686869 3.73737374 3.78787879 3.83838384 3.88888889
 3.93939394 3.98989899 4.04040404 4.09090909 4.14141414 4.19191919
 4.24242424 4.29292929 4.34343434 4.39393939 4.44444444 4.4949